In [3]:
%pwd


'c:\\Users\\dell\\OneDrive\\Desktop\\chatbot\\Medical-chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\dell\\OneDrive\\Desktop\\chatbot\\Medical-chatbot'

In [6]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter


In [7]:
def load_pdf_file(data):
    loader=DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents
    

In [8]:
extracted_data=load_pdf_file(data='C:/Users/dell/OneDrive/Desktop/chatbot/Medical-chatbot/Data/')

In [9]:
#extracted_data

In [10]:
#split the Data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap  = 20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("The number of text chunks created:", len(text_chunks))

The number of text chunks created: 5859


In [12]:


from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings



In [13]:
#download the embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings


In [14]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

embeddings = download_hugging_face_embeddings()
print("Embeddings loaded successfully!")


Embeddings loaded successfully!


In [15]:
query_result=embeddings.embed_query("Hello World")
print("Length:",len(query_result))

Length: 384


In [16]:

#query_result


In [17]:
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

True

In [18]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-0h62md9.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [21]:
#embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)


In [22]:
#load exixting index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [24]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [25]:
retrieved_docs=retriever.invoke("What is Acne?")

In [26]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)

print("Gemini model loaded successfully!")


Gemini model loaded successfully!


In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What are the common treatments for hypertension?"
response = rag_chain.invoke(query)

print(response)

Antihypertensive drugs are commonly used to treat hypertension, with various classes and individual drugs available. These include ACE inhibitors, such as captopril and lisinopril, and calcium channel blocking agents. Dosage varies depending on the patient, drug, and severity of hypertension, and drugs may be used alone or in combination.
